In [0]:
import numpy as np
import pandas as pd
import math
import io

In [2]:
!pip install pgmpy

In [0]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from pgmpy.estimators import K2Score
from pgmpy.inference import VariableElimination
from timeit import default_timer as timer
from pgmpy.estimators import ExhaustiveSearch
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BayesianEstimator

In [4]:
from google.colab import files
uploaded = files.upload()

Saving AND.csv to AND (1).csv
Saving Table2.csv to Table2 (3).csv
Saving Table3.csv to Table3 (3).csv
Saving Table4.csv to Table4 (3).csv
Saving Table5.csv to Table5 (3).csv
Saving Table6.csv to Table6 (3).csv
Saving Table7.csv to Table7 (3).csv
Saving Table8.csv to Table8 (3).csv


# Task 1

# Data Pre-Processing for Marginal Table 

In [5]:
#Reading the Marginal distribution file
read=pd.read_csv("Table2.csv")
# etxracting values from the data frame
marginal=read.values
# removing the extra column
marginal=marginal[:,1:7]
x1=[]
x2=[]
x3=[]
x4=[]
x5=[]
x6=[]
#removing unwanted info and finding the probability percentage
for i in range (0,len(marginal)):
    for j in range (0,len(marginal[0])):
        value=[]
        marginal[i][j]=(float(marginal[i][j].split('%')[0]))/100
        if marginal[i][j]!=0:
            value.append(marginal[i][j])
            #extracting the value of marginal probability
            if j==0:
                x1.append(value)
            elif j==1:
                x2.append(value)
            elif j==2:
                x3.append(value)
            elif j==3:
                x4.append(value)
            elif j==4:
                x5.append(value)
            elif j==5:
                x6.append(value)
print(marginal)

[[0.78 0.275 0.18 0.715 0.375 0.015]
 [0.015 0.32 0.66 0.105 0.11 0.32]
 [0.055 0.025 0.16 0.01 0.105 0.14]
 [0.15 0.17 0.0 0.17 0.41 0.315]
 [0.0 0.21 0.0 0.0 0.0 0.21]]


In [0]:
correlation=np.zeros((6,6))
important=np.zeros((6,6))
cpd_values=dict()

# Data Pre-Processing for CPD

In [0]:
def prep_data(dataset):
    global cpd_values
    #Reading the CPD for given X1
    read=pd.read_csv(dataset)
    y=list(read)
    #y is the given variable
    y=int(y[0].split('x')[1])
    #extracting values
    cpd=read.values
    # loop for fixing individual element
    for i in range(0,len(cpd)):
        for j in range(0,len(cpd[0])):
            # finding the variable name 
            if(j==0 and i!=0):
                temp=cpd[i][j].split(' ')[0]
                cpd[i][j]=int(temp.split('x')[1])
            # finding the probabilty in percentage
            elif(j>0):
                cpd[i][j]=(float(cpd[i][j].split('%')[0]))/100
    #print(cpd)
    # function call for finding corellation
    #returned last key and values of cpd
    key,values=find_correlation(cpd,y)
    # add key value to dict
    cpd_values[key]=values

# Finding the correlation in table

In [0]:
def find_correlation(cpd,y):
    # sum of xi|x1
    sum_x=np.zeros((1,3))
    global independence
    global cpd_values
    # values for the key
    values=[]
    # first x_num
    prev_x_num=cpd[1][0]%10
    #loop for finding values from cpd given xi
    for i in range(1,len(cpd)):
        column=[]
        for j in range(1,len(cpd[0])):
            #finding the xi
            x_num=cpd[i][0]%10
            #finding the xi sub part
            x_sub=int(cpd[i][0]/10)
            # calculating the joint probability
            p_xi_xj=cpd[i][j]*cpd[0][j]
            # calculating the sum of x2|x1
            correlation[x_num-1][y-1]+=abs(p_xi_xj-(cpd[0][j]*marginal[x_sub][x_num-1]))
            #append values for each column
            column.append(cpd[i][j])
        # if x_num changes 
        if prev_x_num!=x_num:
            #add values to dict
            cpd_values[key]=values
            # initailise value for new sub x
            values=[]
            # store the change x_num
            prev_x_num=x_num
        # create key
        key='x'+str(x_num)+'_x'+str(y)
        # add values for the x_num key
        values.append(column)
    return key,values

#  Prepare data and Calculate Correlation

In [67]:
# Function call for preprocessing of each dataset
#print("X1:")
prep_data("Table3.csv")

#print("X2:")
prep_data("Table4.csv")


#print("X3:")
prep_data("Table5.csv")


#print("X4:")
prep_data("Table6.csv")


#print("X5:")
prep_data("Table7.csv")


#print("X6:")
prep_data("Table8.csv")


print(correlation)
'''
for k,v in cpd_values.items():
    print(k)'''

[[0.       0.       0.       0.11957  0.       0.16037 ]
 [0.15977  0.       0.218758 0.1157   0.12939  0.175315]
 [0.       0.218525 0.       0.       0.115965 0.094025]
 [0.11943  0.       0.       0.       0.       0.14307 ]
 [0.       0.12926  0.11552  0.       0.       0.      ]
 [0.160155 0.       0.11258  0.14347  0.       0.      ]]


'\nfor k,v in cpd_values.items():\n    print(k)'

# Finding the Correlation 

In [68]:
for i in range (0,len(important)):
    for j in range(0,len(important[0])):
        if correlation[i][j]<0.15:
            # if less than threshold then less correlation
            important[i][j]=0
        else:
            important[i][j]=1
# converting into data frame
df=pd.DataFrame.from_records(important,columns=['x1','x2','x3','x4','x5','x6'])
print(df)

    x1   x2   x3   x4   x5   x6
0  0.0  0.0  0.0  0.0  0.0  1.0
1  1.0  0.0  1.0  0.0  0.0  1.0
2  0.0  1.0  0.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0  0.0  0.0
4  0.0  0.0  0.0  0.0  0.0  0.0
5  1.0  0.0  0.0  0.0  0.0  0.0


# Creating CPD

In [0]:
def create_cpd(x,y):
    cpd=TabularCPD('x'+str(x),len(cpd_values['x'+str(x)+'_x'+str(y)]),cpd_values['x'+str(x)+'_x'+str(y)],evidence=['x'+str(y)],evidence_card=[len(cpd_values['x'+str(x)+'_x'+str(y)][0])])
    #print(cpd)
    return cpd

In [0]:
# calling function for creating cpd
cpd_x2_x1=create_cpd(2,1)
cpd_x4_x1=create_cpd(4,1)
cpd_x6_x1=create_cpd(6,1)
cpd_x3_x2=create_cpd(3,2)
cpd_x5_x2=create_cpd(5,2)
cpd_x2_x3=create_cpd(2,3)
cpd_x5_x3=create_cpd(5,3)
cpd_x6_x3=create_cpd(6,3)
cpd_x1_x4=create_cpd(1,4)
cpd_x2_x4=create_cpd(2,4)
cpd_x6_x4=create_cpd(6,4)
cpd_x2_x5=create_cpd(2,5)
cpd_x3_x5=create_cpd(3,5)
cpd_x1_x6=create_cpd(1,6)
cpd_x2_x6=create_cpd(2,6)
cpd_x3_x6=create_cpd(3,6)
cpd_x4_x6=create_cpd(4,6)

# Creating Marginal probability

In [0]:
#creating marginal probability for each node
cpd_x1=TabularCPD('x1',4,x1)
cpd_x2=TabularCPD('x2',5,x2)
cpd_x3=TabularCPD('x3',3,x3)
cpd_x4=TabularCPD('x4',4,x4)
cpd_x5=TabularCPD('x5',4,x5)
cpd_x6=TabularCPD('x6',5,x6)

# Task 2

# Create Bayes Model

#Function for sampling the Dataset using CPD

In [0]:
def sampling_model(model,x):
  infer=BayesianModelSampling(model)
  sample=infer.forward_sample(size=1000,return_type='dataframe')
  return sample

# Function for calculating the K2 Score

In [0]:
def K2_scoring(sample,model,x):
  s=K2Score(sample)
  print("K2 score of model"+str(x)+":"+str(s.score(model)))
  return s.score(model)

# Model 1

In [90]:
# create bayesian model
model1= BayesianModel()
# adding edges to the model
model1.add_edges_from([('x6','x4'),('x6','x1'),('x1','x2'),('x2','x5'),('x2','x3')])
# checking for the nodes connected to the given node
print(model1.active_trail_nodes('x6'))
# checking all the nodes in the model
print(model1.edges())
# adding cpds to model
model1.add_cpds(cpd_x6,cpd_x4_x6,cpd_x1_x6,cpd_x2_x1,cpd_x5_x2,cpd_x3_x2)
# checking for cpds in model
model1.get_cpds()
#checking for model 
model1.check_model()

{'x6': set(['x2', 'x3', 'x1', 'x6', 'x4', 'x5'])}
[('x2', 'x3'), ('x2', 'x5'), ('x1', 'x2'), ('x6', 'x1'), ('x6', 'x4')]


True

# Sampling the model

In [0]:
x1=sampling_model(model1,1)

In [92]:
s1=K2_scoring(x1,model1,1)

K2 score of model1:-6388.92059316


# Model 2

In [93]:
#create bayesian model
model2=BayesianModel()
#add edges to the model
model2.add_edges_from([('x1','x6'),('x1','x4'),('x1','x2'),('x2','x5'),('x2','x3')])
# checking for the nodes connected to the given node
print(model2.active_trail_nodes('x1'))
# checking all the nodes in the model
print(model2.edges())
# adding cpds to model
model2.add_cpds(cpd_x1,cpd_x4_x1,cpd_x6_x1,cpd_x2_x1,cpd_x5_x2,cpd_x3_x2)
# checking for cpds in model
model2.get_cpds()
#checking for model 
model2.check_model()

{'x1': set(['x2', 'x3', 'x1', 'x6', 'x4', 'x5'])}
[('x2', 'x3'), ('x2', 'x5'), ('x1', 'x2'), ('x1', 'x6'), ('x1', 'x4')]


True

# Sampling the Model

In [0]:
x2=sampling_model(model2,2)

In [95]:
s2=K2_scoring(x2,model2,2)

K2 score of model2:-6412.07387078


# Model 3

In [96]:
#create bayesian model
model3=BayesianModel()
#add edges to the model
model3.add_edges_from([('x1','x6'),('x1','x4'),('x4','x2'),('x2','x5'),('x6','x3')])
# checking for the nodes connected to the given node
print(model3.active_trail_nodes('x1'))
# checking all the nodes in the model
print(model3.edges())
# adding cpds to model
model3.add_cpds(cpd_x1,cpd_x4_x1,cpd_x6_x1,cpd_x2_x4,cpd_x5_x2,cpd_x3_x6)
# checking for cpds in model
model3.get_cpds()
#checking for model 
model3.check_model()

{'x1': set(['x2', 'x3', 'x1', 'x6', 'x4', 'x5'])}
[('x2', 'x5'), ('x1', 'x6'), ('x1', 'x4'), ('x6', 'x3'), ('x4', 'x2')]


True

# Sampling the model

In [0]:
x3=sampling_model(model3,3)

In [98]:
s3=K2_scoring(x3,model3,3)

K2 score of model3:-6429.4707998


# Model 4

In [99]:
#create bayesian model
model4=BayesianModel()
#add edges to the model
model4.add_edges_from([('x6','x1'),('x6','x2'),('x6','x3'),('x6','x4'),('x2','x5')])
# checking for the nodes connected to the given node
print(model4.active_trail_nodes('x6'))
# checking all the nodes in the model
print(model4.edges())
# adding cpds to model
model4.add_cpds(cpd_x6,cpd_x1_x6,cpd_x2_x6,cpd_x3_x6,cpd_x4_x6,cpd_x5_x2)
# checking for cpds in model
model4.get_cpds()
#checking for model 
model4.check_model()

{'x6': set(['x2', 'x3', 'x1', 'x6', 'x4', 'x5'])}
[('x2', 'x5'), ('x6', 'x2'), ('x6', 'x3'), ('x6', 'x1'), ('x6', 'x4')]


True

# Sampling the model

In [0]:
x4=sampling_model(model4,4)

In [101]:
s4=K2_scoring(x4,model4,4)

K2 score of model4:-6486.50445105


# Model 5

In [102]:
#create bayesian model
model5=BayesianModel()
#add edges to the model
model5.add_edges_from([('x6','x1'),('x1','x2'),('x6','x3'),('x1','x4'),('x3','x5')])
# checking for the nodes connected to the given node
print(model5.active_trail_nodes('x6'))
# checking all the nodes in the model
print(model5.edges())
# adding cpds to model
model5.add_cpds(cpd_x6,cpd_x1_x6,cpd_x2_x1,cpd_x3_x6,cpd_x4_x1,cpd_x5_x3)
# checking for cpds in model
model5.get_cpds()
#checking for model 
model5.check_model()

{'x6': set(['x2', 'x3', 'x1', 'x6', 'x4', 'x5'])}
[('x3', 'x5'), ('x1', 'x2'), ('x1', 'x4'), ('x6', 'x3'), ('x6', 'x1')]


True

# Sampling the Model

In [0]:
x5=sampling_model(model5,5)


In [104]:
s5=K2_scoring(x5,model5,5)

K2 score of model5:-6447.90440184


# Model 6

In [105]:
#create bayesian model
model6=BayesianModel()
#add edges to the model
model6.add_edges_from([('x6','x1'),('x1','x2'),('x6','x3'),('x1','x4'),('x2','x5')])
# checking for the nodes connected to the given node
print(model6.active_trail_nodes('x6'))
# checking all the nodes in the model
print(model6.edges())
# adding cpds to model
model6.add_cpds(cpd_x6,cpd_x1_x6,cpd_x2_x1,cpd_x3_x6,cpd_x4_x1,cpd_x5_x2)
# checking for cpds in model
model6.get_cpds()
#checking for model 
model6.check_model()

{'x6': set(['x2', 'x3', 'x1', 'x6', 'x4', 'x5'])}
[('x2', 'x5'), ('x1', 'x2'), ('x1', 'x4'), ('x6', 'x3'), ('x6', 'x1')]


True

# Sampling the data

In [0]:
x6=sampling_model(model6,6)

In [107]:
s6=K2_scoring(x6,model6,6)

K2 score of model6:-6519.75749316


# Model 7

In [108]:
#create bayesian model
model7=BayesianModel()
#add edges to the model
model7.add_edges_from([('x4','x1'),('x4','x2'),('x1','x6'),('x2','x3'),('x2','x5')])
# checking for the nodes connected to the given node
print(model7.active_trail_nodes('x4'))
# checking all the nodes in the model
print(model7.edges())
# adding cpds to model
model7.add_cpds(cpd_x4,cpd_x1_x4,cpd_x2_x4,cpd_x3_x2,cpd_x6_x1,cpd_x5_x2)
# checking for cpds in model
model7.get_cpds()
#checking for model 
model7.check_model()

{'x4': set(['x2', 'x3', 'x1', 'x6', 'x4', 'x5'])}
[('x2', 'x3'), ('x2', 'x5'), ('x1', 'x6'), ('x4', 'x2'), ('x4', 'x1')]


True

# Sampling the data

In [0]:
x7=sampling_model(model7,7)

In [110]:
s7=K2_scoring(x7,model7,7)

K2 score of model7:-6504.08628778


# Calulating the High probablility TH looks

In [113]:
count=x1.groupby(best_sample.columns.tolist(),as_index=False).size().reset_index().rename(columns={0:'c'})
max_value=count.loc[count['c'].idxmax()]
print("High Probability th is:")
print(max_value)

High Probability th is:
x1     0
x6     3
x4     0
x2     1
x3     1
x5     3
c     23
Name: 146, dtype: int64


# Calulating the low probablility TH looks

In [114]:
count=x1.groupby(best_sample.columns.tolist(),as_index=False).size().reset_index().rename(columns={0:'c'})
min_value=count.loc[count['c'].idxmin()]
print("Low probability th is:")
print(min_value)

Low probability th is:
x1    0
x6    0
x4    0
x2    0
x3    1
x5    3
c     1
Name: 0, dtype: int64


# Task 3

# Converting into Markov Model

In [116]:
mm=model1.to_markov_model()
mm.nodes()
mm.edges()

[('x2', 'x3'), ('x2', 'x1'), ('x2', 'x5'), ('x1', 'x6'), ('x6', 'x4')]

# Inference from the  Markov Model

In [117]:
# creating a map query object
inference = VariableElimination(mm)
# starting the timer
start = timer()
# Querying the model using variable and evidence
phi_query = inference.query(['x6'],evidence={'x1':3})['x6']
#timer end
end = timer()
print(round(end - start,6))
print(phi_query)

0.01026
+------+-----------+
| x6   |   phi(x6) |
+======+===========+
| x6_0 |    0.0000 |
+------+-----------+
| x6_1 |    0.3993 |
+------+-----------+
| x6_2 |    0.2001 |
+------+-----------+
| x6_3 |    0.1999 |
+------+-----------+
| x6_4 |    0.2006 |
+------+-----------+


# Inference from the Bayesian Model

In [118]:
# creating a map query object
inference = VariableElimination(model1)
# starting the timer
start = timer()
# Querying the model using variable and evidence
phi_query = inference.query(['x6'],evidence={'x1':3})['x6']
#timer end
end = timer()
print(round(end - start,6))
print(phi_query)

0.00899
+------+-----------+
| x6   |   phi(x6) |
+======+===========+
| x6_0 |    0.0000 |
+------+-----------+
| x6_1 |    0.3993 |
+------+-----------+
| x6_2 |    0.2001 |
+------+-----------+
| x6_3 |    0.1999 |
+------+-----------+
| x6_4 |    0.2006 |
+------+-----------+


# Task 4

# Reading the dataset

In [0]:
#reading the data
and_data=pd.read_csv("AND.csv")
#removing unwanted columns
and_data=and_data.drop(["Number","img_id"],axis=1)

# Searching for DAG

In [0]:
# find the k2 of the given data
k2=K2Score(and_data)
# Perform Hill Climb Search on the dataset
hc=HillClimbSearch(and_data,scoring_method=k2)

# Finding the Best Model

In [44]:
# Predict the best model
best_model=hc.estimate()
print(best_model.edges())

[('f3', 'f8'), ('f3', 'f9'), ('f3', 'f4'), ('f5', 'f9'), ('f5', 'f3'), ('f9', 'f1'), ('f9', 'f2'), ('f9', 'f4'), ('f9', 'f6'), ('f9', 'f7'), ('f9', 'f8')]


# Constructing the Bayesian Model

In [46]:
# Create Bayesian Model
model=BayesianModel()
# add edges to bayesian model
model.add_edges_from([('f3', 'f4'), ('f3', 'f9'), ('f3', 'f8'), ('f5', 'f9'), ('f5', 'f3'), ('f9', 'f8'), ('f9', 'f7'), ('f9', 'f1'), ('f9', 'f6'), ('f9', 'f2'), ('f9', 'f4')])
#add cpd to bayesian model
model.fit(and_data)
#extract the cpds
model.get_cpds()

[<TabularCPD representing P(f1:4 | f9:2) at 0x7f42b4a7a2d0>,
 <TabularCPD representing P(f2:5 | f9:2) at 0x7f42b4a8bad0>,
 <TabularCPD representing P(f3:3 | f5:4) at 0x7f42b4a76f50>,
 <TabularCPD representing P(f4:5 | f3:3, f9:2) at 0x7f42b4891490>,
 <TabularCPD representing P(f5:4) at 0x7f42b489b2d0>,
 <TabularCPD representing P(f6:4 | f9:2) at 0x7f42b489ba10>,
 <TabularCPD representing P(f7:4 | f9:2) at 0x7f42b4891950>,
 <TabularCPD representing P(f8:5 | f3:3, f9:2) at 0x7f42b4a6d950>,
 <TabularCPD representing P(f9:2 | f3:3, f5:4) at 0x7f42b4a73c90>]

# Calculating th K2 Score

In [48]:
print('K2 Score:'+str(k2.score(model)))

K2 Score:-9462.70489237


# Model2

In [47]:
model2=BayesianModel()
model2.add_edges_from([('f3', 'f4'), ('f3', 'f9'), ('f3', 'f8'), ('f5', 'f9'), ('f9', 'f8'), ('f9', 'f7'), ('f9', 'f1'), ('f9', 'f6'), ('f9', 'f2'), ('f9', 'f4')])
model.fit(and_data)
model2.get_cpds()
print('K2 Score:'+str(k2.score(model2)))

K2 Score:-9476.24220506


# Model 3

In [49]:
model3=BayesianModel()
model3.add_edges_from([('f3', 'f4'), ('f3', 'f9'), ('f3', 'f8'), ('f5', 'f9'), ('f5', 'f3'), ('f9', 'f8'), ('f9', 'f7'), ('f9', 'f1'), ('f9', 'f6'), ('f9', 'f2')])
model.fit(and_data)
model3.get_cpds()
print('K2 Score:'+str(k2.score(model3)))

K2 Score:-9464.57546107


# Model 4

In [50]:
model4=BayesianModel()
model4.add_edges_from([('f3', 'f4'), ('f3', 'f9'), ('f3', 'f8'), ('f5', 'f3'), ('f9', 'f8'), ('f9', 'f7'), ('f9', 'f1'), ('f9', 'f6'), ('f9', 'f2'), ('f9', 'f4')])
model.fit(and_data)
model4.get_cpds()
print('K2 Score:'+str(k2.score(model4)))

K2 Score:-9526.33716368


# Model 5

In [51]:
model5=BayesianModel()
model5.add_edges_from([('f3', 'f4'), ('f3', 'f9'), ('f5', 'f9'), ('f5', 'f3'), ('f9', 'f8'), ('f9', 'f7'), ('f9', 'f1'), ('f9', 'f6'), ('f9', 'f2'), ('f9', 'f4')])
model.fit(and_data)
model5.get_cpds()
print('K2 Score:'+str(k2.score(model5)))

K2 Score:-9463.48683957


# Convert to Markov Model

In [120]:
# best Bayesian to markov conversion
mm=model.to_markov_model()
mm.nodes()
mm.edges()

[('f1', 'f9'),
 ('f2', 'f9'),
 ('f3', 'f8'),
 ('f3', 'f9'),
 ('f3', 'f4'),
 ('f3', 'f5'),
 ('f4', 'f9'),
 ('f5', 'f9'),
 ('f6', 'f9'),
 ('f7', 'f9'),
 ('f8', 'f9')]

# Finding the inference of the AND Markov model

In [123]:
# creating the inference 
inference = VariableElimination(mm)
#start Time
start = timer()
# Query for a variable given evidence
phi_query = inference.query(['f4'],evidence={'f3':0})['f4']
end = timer()
print(round(end - start,6)) 
print(phi_query)

0.027941
+------+-----------+
| f4   |   phi(f4) |
+======+===========+
| f4_0 |    0.1667 |
+------+-----------+
| f4_1 |    0.2348 |
+------+-----------+
| f4_2 |    0.4545 |
+------+-----------+
| f4_3 |    0.1212 |
+------+-----------+
| f4_4 |    0.0227 |
+------+-----------+


# Finding the inference of the Best And Bayesian model

In [122]:
inference = VariableElimination(model)
#start Time
start = timer()
# Query for a variable given evidence
phi_query = inference.query(['f4'],evidence={'f3':0})['f4']
end = timer()
print(round(end - start,6))
print(phi_query)

0.015916
+------+-----------+
| f4   |   phi(f4) |
+======+===========+
| f4_0 |    0.1667 |
+------+-----------+
| f4_1 |    0.2348 |
+------+-----------+
| f4_2 |    0.4545 |
+------+-----------+
| f4_3 |    0.1212 |
+------+-----------+
| f4_4 |    0.0227 |
+------+-----------+
